In [1]:
import pandas as pd;
from pandas import DataFrame, Series;
import datetime;
import json;

In [2]:
import sys
sys.path.insert(0, '../DAL')

import dataService
service = dataService.DataService()
r_r = service.review_dates()

Drill is active: True
started on: 2016-03-27 12:58:47.219968
finished on: 2016-03-27 13:04:14.375254


In [3]:
#Compute date; remove unnecessary columns & reviews
# r_r = DataFrame(dataService.DataService.review_dates()[:100])
lastDay = datetime.datetime(2015, 12, 24)
r_r['reference_date'] = r_r['reference_date'].map(lambda x: (lastDay - datetime.datetime.strptime(x, '%Y-%m-%d')).days)
r_r['review_date'] = r_r['review_date'].map(lambda x: (lastDay - datetime.datetime.strptime(x, '%Y-%m-%d')).days)

In [4]:
r_r.head()

,reference_id,reference_date,authority_stars,authority_id,review_id,review_date,user_id,validation_token
0,d3w57QihQsYNDmZi3x1x8w,507,3,9OZH1Ecw-qUkCW5MS0NefA,fpJ_Bgho7G3E-_WWaMJpjA,2029,YKDw2WGj-MQjz7eRsb5FHg,9OZH1Ecw-qUkCW5MS0NefAYKDw2WGj-MQjz7eRsb5FHg
1,d3w57QihQsYNDmZi3x1x8w,507,3,9OZH1Ecw-qUkCW5MS0NefA,nuZlz-65cXIsLZPGk6EwDw,1807,_CtkjL-e6O_lMShkN6HuSA,9OZH1Ecw-qUkCW5MS0NefA_CtkjL-e6O_lMShkN6HuSA
2,d3w57QihQsYNDmZi3x1x8w,507,3,9OZH1Ecw-qUkCW5MS0NefA,DtWoMT9VcFX9ycdZFpZoow,1510,0kRiaCs6damnnCpjPNOqRw,9OZH1Ecw-qUkCW5MS0NefA0kRiaCs6damnnCpjPNOqRw
3,d3w57QihQsYNDmZi3x1x8w,507,3,9OZH1Ecw-qUkCW5MS0NefA,M28dSKnyInjQu0I9NJlfig,1000,RJgfj-6Uq-lyiPJJcyCVlQ,9OZH1Ecw-qUkCW5MS0NefARJgfj-6Uq-lyiPJJcyCVlQ
4,d3w57QihQsYNDmZi3x1x8w,507,3,9OZH1Ecw-qUkCW5MS0NefA,QRRumDA7QEUGwQ2sD_KswA,927,fTg6Rz2Rs1LLYN_U7TJbcQ,9OZH1Ecw-qUkCW5MS0NefAfTg6Rz2Rs1LLYN_U7TJbcQ


In [5]:
r_r['days_since_review'] = r_r.apply(lambda x: x['reference_date'] - x['review_date'], axis=1)
r_r['time_distribution'] = r_r.apply(
    lambda x: 1 if (x['days_since_review'] > 0) 
    else (
        0 if (x['days_since_review'] + x['reference_date'] >= 0)
        else -1
    ), axis=1)
# then -1)
r_r = r_r[['authority_id', 'reference_id', 'user_id', 'review_id', 'authority_stars', 'time_distribution', 'validation_token']]
r_r = DataFrame(r_r.loc[r_r['time_distribution'] >= 0])

In [6]:
#Add distances
u_u = pd.read_json('../full_dataset/usergraph_vt_resto_food_chinese.json')
r_r = pd.merge(r_r, u_u, on='validation_token')
u_u = None

In [7]:
r_r.head()

,authority_id,reference_id,user_id,review_id,authority_stars,time_distribution,validation_token,distance
0,9OZH1Ecw-qUkCW5MS0NefA,d3w57QihQsYNDmZi3x1x8w,RJgfj-6Uq-lyiPJJcyCVlQ,M28dSKnyInjQu0I9NJlfig,3,0,9OZH1Ecw-qUkCW5MS0NefARJgfj-6Uq-lyiPJJcyCVlQ,3
1,9OZH1Ecw-qUkCW5MS0NefA,d3w57QihQsYNDmZi3x1x8w,fTg6Rz2Rs1LLYN_U7TJbcQ,QRRumDA7QEUGwQ2sD_KswA,3,0,9OZH1Ecw-qUkCW5MS0NefAfTg6Rz2Rs1LLYN_U7TJbcQ,5
2,9OZH1Ecw-qUkCW5MS0NefA,d3w57QihQsYNDmZi3x1x8w,9OZH1Ecw-qUkCW5MS0NefA,d3w57QihQsYNDmZi3x1x8w,3,0,9OZH1Ecw-qUkCW5MS0NefA9OZH1Ecw-qUkCW5MS0NefA,2
3,9OZH1Ecw-qUkCW5MS0NefA,poFXO3tkqHW3xQAv9qxvxw,9OZH1Ecw-qUkCW5MS0NefA,poFXO3tkqHW3xQAv9qxvxw,1,0,9OZH1Ecw-qUkCW5MS0NefA9OZH1Ecw-qUkCW5MS0NefA,2
4,9OZH1Ecw-qUkCW5MS0NefA,aD7ArFpj1D72KkVoj0WcdQ,9OZH1Ecw-qUkCW5MS0NefA,aD7ArFpj1D72KkVoj0WcdQ,3,0,9OZH1Ecw-qUkCW5MS0NefA9OZH1Ecw-qUkCW5MS0NefA,2


In [8]:
distribution = r_r.groupby(['authority_id', 'time_distribution', 'distance', 'authority_stars']).size().reset_index()

In [9]:
distribution.to_json('../full_dataset/visit_distribution.json')